In [2]:
import pandas as pd
train_final = pd.read_csv('train_final.csv')
#train_final.dropna(subset = ["sales"], inplace=True)


import pandas as pd
test_data = pd.read_csv('test_data.csv')
test_data['sales']=-1

result_data = pd.concat([train_final, test_data], axis=0)

In [4]:
import pandas as pd
sku_final = pd.read_csv('sku_final.csv')
sku_final = sku_final.drop(['commodity_group_caption_RU','commodity_group_caption_UKR','productType_caption_RU','productType_caption_UKR','productCategory_caption_UKR','productCategory_caption_RU'], axis=1)
sku_final = sku_final.drop(['countryOfOrigin','productType_caption_ENG','lagerUnitType_caption'], axis=1)
sku_final = sku_final.drop(['commodity_group','productCategoryId'], axis=1)
sku_final = sku_final.drop(['productCategory_caption_ENG'], axis=1)
sku_final = sku_final.drop(['countryOfOrigin_caption'], axis=1)
sku_final = sku_final.drop(['lagerUnitTypeId','lagerUnitQuantity'], axis=1)
sku_final = sku_final.drop(['brandId','trademark'], axis=1)

result_data = pd.merge(result_data, sku_final, how="left", on=["SKU","SKU"])

#result = pd.concat([result_data, pd.get_dummies(result_data['commodity_group_caption_ENG'])], axis=1)
result = result_data.drop(['commodity_group_caption_ENG'], axis=1)
result = result.drop(['productTypeId'], axis=1)

geo_df = pd.read_csv('geo_params.csv')

In [ ]:
import pandas as pd
import datetime
from catboost import CatBoostRegressor
from sklearn.feature_selection import RFECV
import numpy as np
import statsmodels.api as sm

def mean_absolute_percentage_error_(y_true, y_pred): 
    return np.sum(np.abs(y_true - y_pred) / np.sum(y_true))

geoCluster_mass = result['geoCluster'].unique()   
for i in range(0,len(geoCluster_mass)):
    mass = result.loc[(result_data['geoCluster'] == geoCluster_mass[i])]
    data_dollar = pd.read_excel('dollar.xlsx')[1:]
    data_dollar.dropna(subset = ["Euro"], inplace=True)
    data_dollar=data_dollar.fillna(0)
    data_dollar=data_dollar.rename(columns={'Data': 'date'})
    data_dollar['date'] = data_dollar['date'].astype(str)
    outer_merged = pd.merge(mass, data_dollar, how="left", on=["date","date"])
    merged_final = outer_merged
    new_cols = ['is_monday', 'is_tuesday', 'is_wednesday', 'is_thursday', 'is_friday', 'is_saturday', 'is_sunday']
    
    for index, col_name in enumerate(new_cols):
        merged_final[col_name] = [datetime.datetime.strptime(date_str, '%Y-%m-%d').weekday() == index for date_str in merged_final.date]
    merged_final=merged_final.fillna(0)

    anomaly_cluster = 681
    merged_final = merged_final[merged_final.geoCluster != anomaly_cluster]
    merged_geo = pd.merge(merged_final, geo_df, how='left', on=['geoCluster', 'geoCluster'])
    
    result1 = merged_geo
    train_base = result1[result1.sales != -1]
    test_base = result1[result1.sales == -1]
    
    y = train_base['sales']
    X = train_base.drop(['sales'], axis=1)
    model = CatBoostRegressor(iterations=250, depth=16, learning_rate=0.8, loss_function="RMSE",)
    model.fit(X.drop(['ID','date'], axis=1), y) 
    y_ = model.predict(X.drop(['ID','date'], axis=1))
    print(mean_absolute_percentage_error_(y, y_))
    print(i)    

    test_base_id = test_base[['ID','sales']]
    prediction = model.predict(test_base.drop(['ID','date','sales'], axis=1))
    test_base_id['sales'] = prediction
    test_data = pd.merge(test_data, test_base_id, how="left", on=["ID","ID"])

0:	learn: 0.1978983	total: 1.7ms	remaining: 40.7ms
1:	learn: 0.1560448	total: 3.17ms	remaining: 36.4ms
2:	learn: 0.1232904	total: 13.9ms	remaining: 102ms
3:	learn: 0.0985265	total: 16.2ms	remaining: 84.9ms
4:	learn: 0.0792565	total: 181ms	remaining: 722ms
5:	learn: 0.0629917	total: 186ms	remaining: 588ms
6:	learn: 0.0509075	total: 222ms	remaining: 572ms
7:	learn: 0.0375487	total: 224ms	remaining: 475ms
8:	learn: 0.0298241	total: 242ms	remaining: 430ms
9:	learn: 0.0238913	total: 252ms	remaining: 378ms
10:	learn: 0.0196979	total: 253ms	remaining: 322ms
11:	learn: 0.0157991	total: 272ms	remaining: 295ms
12:	learn: 0.0126972	total: 278ms	remaining: 256ms
13:	learn: 0.0102241	total: 287ms	remaining: 225ms
14:	learn: 0.0082627	total: 328ms	remaining: 219ms
15:	learn: 0.0067136	total: 337ms	remaining: 189ms
16:	learn: 0.0054757	total: 500ms	remaining: 235ms
17:	learn: 0.0044575	total: 823ms	remaining: 320ms
18:	learn: 0.0037086	total: 899ms	remaining: 284ms
19:	learn: 0.0028034	total: 916ms	r

<ipython-input-9-6447296013be>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_base_id['sales'] = prediction


0:	learn: 0.2789168	total: 964us	remaining: 23.1ms
1:	learn: 0.2184245	total: 25.8ms	remaining: 296ms
2:	learn: 0.1694516	total: 26.8ms	remaining: 196ms
3:	learn: 0.1419068	total: 28ms	remaining: 147ms
4:	learn: 0.1170668	total: 31.4ms	remaining: 126ms
5:	learn: 0.1005028	total: 32.6ms	remaining: 103ms
6:	learn: 0.0798867	total: 137ms	remaining: 353ms
7:	learn: 0.0639086	total: 149ms	remaining: 316ms
8:	learn: 0.0510875	total: 201ms	remaining: 358ms
9:	learn: 0.0407167	total: 226ms	remaining: 339ms
10:	learn: 0.0326019	total: 330ms	remaining: 421ms
11:	learn: 0.0260883	total: 547ms	remaining: 593ms
12:	learn: 0.0209007	total: 572ms	remaining: 528ms
13:	learn: 0.0167149	total: 575ms	remaining: 452ms
14:	learn: 0.0134051	total: 577ms	remaining: 384ms
15:	learn: 0.0107882	total: 579ms	remaining: 326ms
16:	learn: 0.0087231	total: 581ms	remaining: 273ms
17:	learn: 0.0071000	total: 583ms	remaining: 227ms
18:	learn: 0.0058328	total: 586ms	remaining: 185ms
19:	learn: 0.0048534	total: 588ms	rem

9:	learn: 0.0646236	total: 1.45s	remaining: 2.17s
10:	learn: 0.0534023	total: 1.48s	remaining: 1.88s
11:	learn: 0.0431636	total: 1.7s	remaining: 1.84s
12:	learn: 0.0346430	total: 2.15s	remaining: 1.99s
13:	learn: 0.0278264	total: 2.59s	remaining: 2.03s
14:	learn: 0.0223641	total: 3.03s	remaining: 2.02s
15:	learn: 0.0180447	total: 3.46s	remaining: 1.95s
16:	learn: 0.0145964	total: 3.9s	remaining: 1.83s
17:	learn: 0.0117162	total: 4.33s	remaining: 1.68s
18:	learn: 0.0094514	total: 4.77s	remaining: 1.51s
19:	learn: 0.0076444	total: 5.21s	remaining: 1.3s
20:	learn: 0.0062552	total: 5.65s	remaining: 1.08s
21:	learn: 0.0051602	total: 6.09s	remaining: 830ms
22:	learn: 0.0043197	total: 6.53s	remaining: 568ms
23:	learn: 0.0036734	total: 6.97s	remaining: 290ms
24:	learn: 0.0032029	total: 7.41s	remaining: 0us
0.01471110811348834
7
0:	learn: 0.0189945	total: 393us	remaining: 9.45ms
1:	learn: 0.0151875	total: 957us	remaining: 11ms
2:	learn: 0.0121491	total: 1.55ms	remaining: 11.3ms
3:	learn: 0.0097

16:	learn: 0.2601802	total: 5.35s	remaining: 2.52s
17:	learn: 0.2394940	total: 5.78s	remaining: 2.25s
18:	learn: 0.2238242	total: 6.22s	remaining: 1.96s
19:	learn: 0.2181000	total: 6.67s	remaining: 1.67s
20:	learn: 0.2155271	total: 7.11s	remaining: 1.35s
21:	learn: 0.1992109	total: 7.55s	remaining: 1.03s
22:	learn: 0.1808547	total: 7.99s	remaining: 695ms
23:	learn: 0.1677458	total: 8.43s	remaining: 351ms
24:	learn: 0.1677056	total: 8.43s	remaining: 0us
0.10557741171908883
13
0:	learn: 0.5599659	total: 15.2ms	remaining: 365ms
1:	learn: 0.4399061	total: 127ms	remaining: 1.46s
2:	learn: 0.3174759	total: 154ms	remaining: 1.13s
3:	learn: 0.2521052	total: 179ms	remaining: 942ms
4:	learn: 0.1982807	total: 181ms	remaining: 722ms
5:	learn: 0.1631928	total: 182ms	remaining: 577ms
6:	learn: 0.1335115	total: 184ms	remaining: 472ms
7:	learn: 0.1083300	total: 196ms	remaining: 417ms
8:	learn: 0.0874569	total: 255ms	remaining: 453ms
9:	learn: 0.0702523	total: 262ms	remaining: 393ms
10:	learn: 0.056439

...

13:	learn: 0.2107250	total: 32.9s	remaining: 25.9s
14:	learn: 0.2067349	total: 35.4s	remaining: 23.6s
15:	learn: 0.2022433	total: 37.9s	remaining: 21.3s
16:	learn: 0.1957109	total: 40.4s	remaining: 19s
17:	learn: 0.1927829	total: 42.9s	remaining: 16.7s
18:	learn: 0.1864199	total: 45.4s	remaining: 14.3s
19:	learn: 0.1814517	total: 47.9s	remaining: 12s
20:	learn: 0.1786344	total: 50.4s	remaining: 9.59s
21:	learn: 0.1773602	total: 52.9s	remaining: 7.21s
22:	learn: 0.1729555	total: 55.4s	remaining: 4.81s
23:	learn: 0.1694175	total: 57.9s	remaining: 2.41s
24:	learn: 0.1656400	total: 1m	remaining: 0us
0.27025545478848984
309
0:	learn: 0.3509326	total: 1.13ms	remaining: 27.2ms
1:	learn: 0.2115699	total: 2.37ms	remaining: 27.3ms
2:	learn: 0.1675492	total: 45.6ms	remaining: 334ms
3:	learn: 0.1351757	total: 86.1ms	remaining: 452ms
4:	learn: 0.1090351	total: 449ms	remaining: 1.8s
5:	learn: 0.0898380	total: 455ms	remaining: 1.44s
6:	learn: 0.0715302	total: 627ms	remaining: 1.61s
7:	learn: 0.058992

23:	learn: 0.2419990	total: 57.5s	remaining: 2.4s
24:	learn: 0.2391804	total: 60s	remaining: 0us
0.3147075671151809
315
0:	learn: 0.4170013	total: 1.66ms	remaining: 39.8ms
1:	learn: 0.3324752	total: 2.94ms	remaining: 33.8ms
2:	learn: 0.2658157	total: 3.54ms	remaining: 25.9ms
3:	learn: 0.2126507	total: 4.78ms	remaining: 25.1ms
4:	learn: 0.1358580	total: 5.31ms	remaining: 21.2ms
5:	learn: 0.1087384	total: 6.31ms	remaining: 20ms
6:	learn: 0.0870686	total: 7.1ms	remaining: 18.3ms
7:	learn: 0.0697436	total: 7.75ms	remaining: 16.5ms
8:	learn: 0.0558863	total: 8.82ms	remaining: 15.7ms
9:	learn: 0.0448413	total: 9.42ms	remaining: 14.1ms
10:	learn: 0.0360297	total: 10.2ms	remaining: 13ms
11:	learn: 0.0289483	total: 11.1ms	remaining: 12.1ms
12:	learn: 0.0232910	total: 12.8ms	remaining: 11.8ms
13:	learn: 0.0186477	total: 14.8ms	remaining: 11.7ms
14:	learn: 0.0149257	total: 30.8ms	remaining: 20.5ms
15:	learn: 0.0119579	total: 94.5ms	remaining: 53.1ms
16:	learn: 0.0095829	total: 98.3ms	remaining: 4

8:	learn: 0.2121850	total: 22.7s	remaining: 40.3s
9:	learn: 0.2031456	total: 25.2s	remaining: 37.8s
10:	learn: 0.1963885	total: 27.7s	remaining: 35.2s
11:	learn: 0.1870707	total: 30.2s	remaining: 32.7s
12:	learn: 0.1818057	total: 32.7s	remaining: 30.2s
13:	learn: 0.1760775	total: 35.2s	remaining: 27.7s
14:	learn: 0.1704504	total: 37.7s	remaining: 25.1s
15:	learn: 0.1631573	total: 40.2s	remaining: 22.6s
16:	learn: 0.1600910	total: 42.7s	remaining: 20.1s
17:	learn: 0.1572774	total: 45.2s	remaining: 17.6s
18:	learn: 0.1548298	total: 47.9s	remaining: 15.1s
19:	learn: 0.1511917	total: 50.4s	remaining: 12.6s
20:	learn: 0.1458625	total: 53s	remaining: 10.1s
21:	learn: 0.1416720	total: 55.5s	remaining: 7.57s
22:	learn: 0.1363669	total: 58s	remaining: 5.05s
23:	learn: 0.1351096	total: 1m	remaining: 2.52s
24:	learn: 0.1318247	total: 1m 3s	remaining: 0us
0.2491742943078752
322
0:	learn: 0.3617920	total: 10.3ms	remaining: 248ms
1:	learn: 0.2904984	total: 323ms	remaining: 3.71s
2:	learn: 0.2190686	

18:	learn: 0.3134888	total: 39s	remaining: 12.3s
19:	learn: 0.3020358	total: 41.5s	remaining: 10.4s
20:	learn: 0.2966393	total: 44.1s	remaining: 8.39s
21:	learn: 0.2903180	total: 46.6s	remaining: 6.35s
22:	learn: 0.2828122	total: 49.1s	remaining: 4.27s
23:	learn: 0.2753705	total: 51.7s	remaining: 2.15s
24:	learn: 0.2674702	total: 54.2s	remaining: 0us
0.3309288665895097
328
0:	learn: 0.8966777	total: 2.56s	remaining: 1m 1s
1:	learn: 0.8059950	total: 5.06s	remaining: 58.2s
2:	learn: 0.7657242	total: 7.58s	remaining: 55.6s
3:	learn: 0.7359834	total: 10.1s	remaining: 53.1s
4:	learn: 0.7136246	total: 12.6s	remaining: 50.6s
5:	learn: 0.6985392	total: 15.2s	remaining: 48s
6:	learn: 0.6836907	total: 17.7s	remaining: 45.5s
7:	learn: 0.6735544	total: 20.2s	remaining: 42.9s
8:	learn: 0.6587482	total: 22.7s	remaining: 40.4s
9:	learn: 0.6480362	total: 25.3s	remaining: 37.9s
10:	learn: 0.6382367	total: 27.8s	remaining: 35.3s
11:	learn: 0.6323517	total: 30.3s	remaining: 32.8s
12:	learn: 0.6196035	tot

3:	learn: 0.7967335	total: 7.96s	remaining: 41.8s
4:	learn: 0.7810114	total: 10.5s	remaining: 42.1s
5:	learn: 0.7630041	total: 13s	remaining: 41.3s
6:	learn: 0.7505990	total: 15.6s	remaining: 40s
7:	learn: 0.7343960	total: 18.1s	remaining: 38.4s
8:	learn: 0.7218777	total: 20.6s	remaining: 36.7s
9:	learn: 0.7085885	total: 23.2s	remaining: 34.8s
10:	learn: 0.6998711	total: 25.8s	remaining: 32.8s
11:	learn: 0.6840885	total: 28.3s	remaining: 30.7s
12:	learn: 0.6696054	total: 30.9s	remaining: 28.5s
13:	learn: 0.6606571	total: 33.4s	remaining: 26.3s
14:	learn: 0.6506252	total: 36s	remaining: 24s
15:	learn: 0.6463341	total: 38.5s	remaining: 21.7s
16:	learn: 0.6370991	total: 41.1s	remaining: 19.3s
17:	learn: 0.6317996	total: 43.6s	remaining: 17s
18:	learn: 0.6202680	total: 46.2s	remaining: 14.6s
19:	learn: 0.6134597	total: 48.7s	remaining: 12.2s
20:	learn: 0.6071567	total: 51.3s	remaining: 9.76s
21:	learn: 0.6035746	total: 53.8s	remaining: 7.34s
22:	learn: 0.5961171	total: 56.3s	remaining: 4.9

In [ ]:
test_data.to_csv('out1.csv',index=False)

In [ ]:
import pandas as pd
data_dollar = pd.read_excel('dollar.xlsx')[1:]
data_dollar.dropna(subset = ["Euro"], inplace=True)
data_dollar=data_dollar.fillna(0)
data_dollar=data_dollar.rename(columns={'Data': 'date'})
data_dollar['date'] = data_dollar['date'].astype(str)
outer_merged = pd.merge(result_data, data_dollar, how="left", on=["date","date"])

In [ ]:
import pandas as pd
sku_final = pd.read_csv('sku_final.csv')
merged_final = pd.merge(outer_merged, sku_final, how="left", on=["SKU","SKU"])

In [ ]:
merged_final = merged_final.drop(['commodity_group_caption_RU','commodity_group_caption_UKR','productType_caption_RU','productType_caption_UKR','productCategory_caption_UKR','productCategory_caption_RU'], axis=1)
merged_final = merged_final.drop(['countryOfOrigin','productType_caption_ENG','Year','lagerUnitType_caption'], axis=1)
merged_final = merged_final.drop(['commodity_group','productCategoryId'], axis=1)
merged_final = merged_final.drop(['productCategory_caption_ENG'], axis=1)
merged_final = merged_final.drop(['countryOfOrigin_caption'], axis=1)
merged_final = merged_final.sort_values(by=['date'])

In [ ]:
from sklearn.preprocessing import LabelEncoder

cat_labels = ['commodity_group_caption_ENG','geoCluster']
merged_final[cat_labels] = merged_final[cat_labels].fillna('NaN')

le = LabelEncoder()
label_encoded_df = merged_final.copy()
for col in cat_labels:
    label_encoded_df[col]=le.fit_transform(label_encoded_df[col])

In [ ]:
import datetime

merged_final = label_encoded_df
new_cols = ['is_monday', 'is_tuesday', 'is_wednesday', 'is_thursday', 'is_friday', 'is_saturday', 'is_sunday']
for index, col_name in enumerate(new_cols):
    merged_final[col_name] = [datetime.datetime.strptime(date_str, '%Y-%m-%d').weekday() == index for date_str in merged_final.date]

In [ ]:
result = pd.concat([merged_final, pd.get_dummies(merged_final['commodity_group_caption_ENG'])], axis=1)
result = result.drop(['commodity_group_caption_ENG'], axis=1)

In [44]:
test_base = result[result.sales == 0]
train_base = result[result.sales != 0]

y = train_base['sales']
X = train_base.drop(['sales'], axis=1)

# Categorical boosting

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(result, y, random_state=42)

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
from catboost import CatBoostRegressor
from sklearn.feature_selection import RFECV
import numpy as np

def mean_absolute_percentage_error_(y_true, y_pred): 
    return np.sum(np.abs(y_true - y_pred) / np.sum(y_true))

model = CatBoostRegressor(iterations=15, depth=16, learning_rate=1, loss_function="RMSE",)
#model.fit(X.drop(['ID'], axis=1), y)

rfecv = RFECV(estimator = model,cv = 250000,scoring = 'neg_mean_absolute_percentage_error')
rfecv.fit(X.drop(['ID'], axis=1), y)

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

0:	learn: 3.1040067	total: 2.76s	remaining: 38.6s
1:	learn: 3.0472329	total: 5.2s	remaining: 33.8s
2:	learn: 3.0156002	total: 7.67s	remaining: 30.7s
3:	learn: 2.9745176	total: 10.2s	remaining: 28s
4:	learn: 2.9565490	total: 12.7s	remaining: 25.3s
5:	learn: 2.9470363	total: 15.2s	remaining: 22.8s
6:	learn: 2.8908771	total: 17.6s	remaining: 20.1s
7:	learn: 2.8775713	total: 20s	remaining: 17.5s
8:	learn: 2.8403307	total: 22.4s	remaining: 14.9s
9:	learn: 2.8223852	total: 24.9s	remaining: 12.4s
10:	learn: 2.8028115	total: 27.3s	remaining: 9.93s
11:	learn: 2.7948962	total: 29.8s	remaining: 7.45s
12:	learn: 2.7855128	total: 32.2s	remaining: 4.95s
13:	learn: 2.7707462	total: 34.8s	remaining: 2.48s
14:	learn: 2.7622066	total: 37.1s	remaining: 0us
0:	learn: 3.0966970	total: 2.57s	remaining: 36s
1:	learn: 3.0553166	total: 5.06s	remaining: 32.9s
2:	learn: 3.0207479	total: 7.48s	remaining: 29.9s
3:	learn: 2.9852801	total: 10s	remaining: 27.5s
4:	learn: 2.9530066	total: 12.7s	remaining: 25.4s
5:	lea

In [75]:
y_ = model.predict(X.drop(['ID'], axis=1))
mean_absolute_percentage_error_(y, y_)

CatBoostError: There is no trained model to use predict(). Use fit() to train model. Then use this method.

# Prediction

In [68]:
y_ans = model.predict(test_base.drop(['sales','ID'], axis=1))

df_answer = pd.DataFrame(columns=['ID','sales'])
df_answer['sales'] = y_ans
df_answer['ID'] = test_base['ID'].values

for i in range(0,len(df_answer)):
    if df_answer.sales[i] < 0:
        df_answer.sales[i] = 0

df_answer

C:\Users\lysyi\AppData\Local\Temp/ipykernel_15864/957978332.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_answer.sales[i] = 0


,ID,sales
0,RR2653982,2.860516
1,RR1835556,0.640024
2,RR2653996,3.196059
3,RR2653968,2.419997
4,RR2653954,0.000000
...,...,...
1666023,RR2814995,0.000000
1666024,RR1795291,3.001508
1666025,RR2815009,3.846124
1666026,RR2815037,1.115777


In [69]:
df_answer.to_csv('out.csv',index=False)

In [9]:
import pandas as pd
data = pd.read_csv('train_final.csv')
data.dropna(subset = ["sales"], inplace=True)
data.to_csv('clean_data.csv')

In [ ]:
data.reset_index(drop=True, inplace=True)
data['date'] = data['date'].astype(str)

In [60]:
import pandas as pd
data_dollar = pd.read_excel('dollar.xlsx')[1:]
data_dollar.dropna(subset = ["Euro"], inplace=True)
data_dollar=data_dollar.fillna(0)
data_dollar=data_dollar.rename(columns={'Data': 'date'})
data_dollar['date'] = data_dollar['date'].astype(str)
outer_merged = pd.merge(data, data_dollar, how="left", on=["date","date"])
outer_merged

In [67]:
import pandas as pd
sku_final = pd.read_csv('sku_final.csv')
outer_merged = pd.read_csv('outer_merged.csv')
merged_final = pd.merge(outer_merged, sku_final, how="left", on=["SKU","SKU"])
merged_final.to_csv('merged_final.csv')

In [73]:
merged_final = merged_final.drop(['commodity_group_caption_RU','commodity_group_caption_UKR','productType_caption_RU','productType_caption_UKR','productCategory_caption_UKR','productCategory_caption_RU'], axis=1)
merged_final = merged_final.drop(['countryOfOrigin','productType_caption_ENG','Year','lagerUnitType_caption'], axis=1)
merged_final = merged_final.drop(['commodity_group','productCategoryId'], axis=1)
merged_final = merged_final.drop(['productCategory_caption_ENG'], axis=1)

In [100]:
merged_final = merged_final.drop(['countryOfOrigin_caption'], axis=1)

In [104]:
merged_final = merged_final.sort_values(by=['date'])

In [105]:
merged_final

,ID,geoCluster,SKU,date,price,sales,Карантин,USD,Euro,Выборы,productTypeId,brandId,lagerUnitQuantity,lagerUnitTypeId,trademark,commodity_group_caption_ENG
1653540,RR52304931,2065,507343,2020-01-03,7.19,6.0,0.0,23.6862,26.4220,0,4699.0,8274.0,70.00,1,NaN,Bakery
1653541,RR52304936,2065,507343,2020-01-08,7.19,4.0,0.0,23.6785,26.3932,0,4699.0,8274.0,70.00,1,NaN,Bakery
1045940,RR43963070,2065,63037,2020-01-08,14.99,3.0,0.0,23.6785,26.3932,0,3467.0,1241.0,0.50,3,1323.0,Mineral water
1045989,RR43964553,2065,369991,2020-01-08,16.59,2.0,0.0,23.6785,26.3932,0,3465.0,1241.0,0.75,3,1323.0,Mineral water
1653542,RR52304937,2065,507343,2020-01-09,7.19,1.0,0.0,23.8345,26.4909,0,4699.0,8274.0,70.00,1,NaN,Bakery
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1148294,RR45619916,2482,233272,2021-07-19,34.19,2.0,0.0,27.2301,32.1410,0,3454.0,2737.0,1.00,3,1839.0,Mineral water
1974295,RR55877425,2992,556482,2021-07-19,11.39,4.0,0.0,27.2301,32.1410,0,4699.0,8274.0,85.00,1,4043.0,Bakery
1148107,RR45618422,2482,67802,2021-07-19,20.79,4.0,0.0,27.2301,32.1410,0,3469.0,1330.0,2.00,3,2781.0,Mineral water
1148508,RR45623020,2482,474025,2021-07-19,20.79,7.0,0.0,27.2301,32.1410,0,3470.0,1330.0,2.00,3,2781.0,Mineral water


In [132]:
merged_final.to_csv('merged_final_.csv')

In [57]:
import pandas as pd
merged_final = pd.read_csv('merged_final_.csv')

In [58]:
merged_final

,Unnamed: 0,ID,geoCluster,SKU,date,price,sales,Карантин,USD,Euro,Выборы,productTypeId,brandId,lagerUnitQuantity,lagerUnitTypeId,trademark,commodity_group_caption_ENG
0,1653540,RR52304931,2065,507343,2020-01-03,7.19,6.0,0.0,23.6862,26.4220,0,4699.0,8274.0,70.00,1,NaN,Bakery
1,1653541,RR52304936,2065,507343,2020-01-08,7.19,4.0,0.0,23.6785,26.3932,0,4699.0,8274.0,70.00,1,NaN,Bakery
2,1045940,RR43963070,2065,63037,2020-01-08,14.99,3.0,0.0,23.6785,26.3932,0,3467.0,1241.0,0.50,3,1323.0,Mineral water
3,1045989,RR43964553,2065,369991,2020-01-08,16.59,2.0,0.0,23.6785,26.3932,0,3465.0,1241.0,0.75,3,1323.0,Mineral water
4,1653542,RR52304937,2065,507343,2020-01-09,7.19,1.0,0.0,23.8345,26.4909,0,4699.0,8274.0,70.00,1,NaN,Bakery
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1981382,1148294,RR45619916,2482,233272,2021-07-19,34.19,2.0,0.0,27.2301,32.1410,0,3454.0,2737.0,1.00,3,1839.0,Mineral water
1981383,1974295,RR55877425,2992,556482,2021-07-19,11.39,4.0,0.0,27.2301,32.1410,0,4699.0,8274.0,85.00,1,4043.0,Bakery
1981384,1148107,RR45618422,2482,67802,2021-07-19,20.79,4.0,0.0,27.2301,32.1410,0,3469.0,1330.0,2.00,3,2781.0,Mineral water
1981385,1148508,RR45623020,2482,474025,2021-07-19,20.79,7.0,0.0,27.2301,32.1410,0,3470.0,1330.0,2.00,3,2781.0,Mineral water


In [59]:
data1 = merged_final.drop(['ID'], axis=1)

In [60]:
data1

,Unnamed: 0,geoCluster,SKU,date,price,sales,Карантин,USD,Euro,Выборы,productTypeId,brandId,lagerUnitQuantity,lagerUnitTypeId,trademark,commodity_group_caption_ENG
0,1653540,2065,507343,2020-01-03,7.19,6.0,0.0,23.6862,26.4220,0,4699.0,8274.0,70.00,1,NaN,Bakery
1,1653541,2065,507343,2020-01-08,7.19,4.0,0.0,23.6785,26.3932,0,4699.0,8274.0,70.00,1,NaN,Bakery
2,1045940,2065,63037,2020-01-08,14.99,3.0,0.0,23.6785,26.3932,0,3467.0,1241.0,0.50,3,1323.0,Mineral water
3,1045989,2065,369991,2020-01-08,16.59,2.0,0.0,23.6785,26.3932,0,3465.0,1241.0,0.75,3,1323.0,Mineral water
4,1653542,2065,507343,2020-01-09,7.19,1.0,0.0,23.8345,26.4909,0,4699.0,8274.0,70.00,1,NaN,Bakery
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1981382,1148294,2482,233272,2021-07-19,34.19,2.0,0.0,27.2301,32.1410,0,3454.0,2737.0,1.00,3,1839.0,Mineral water
1981383,1974295,2992,556482,2021-07-19,11.39,4.0,0.0,27.2301,32.1410,0,4699.0,8274.0,85.00,1,4043.0,Bakery
1981384,1148107,2482,67802,2021-07-19,20.79,4.0,0.0,27.2301,32.1410,0,3469.0,1330.0,2.00,3,2781.0,Mineral water
1981385,1148508,2482,474025,2021-07-19,20.79,7.0,0.0,27.2301,32.1410,0,3470.0,1330.0,2.00,3,2781.0,Mineral water


In [61]:
y = data1['sales']
X = data1.drop(['sales'], axis=1)

In [118]:
#from statsmodels.tsa.seasonal import seasonal_decompose
#season = seasonal_decompose(clean_data['y_total_trans'][0:106].values, model='multiplicative', period=53)
#seson_var = (season.seasonal - 1)[0:53].tolist()
#clean_data['season'] = (seson_var+seson_var+seson_var+seson_var)[0:len(clean_data['y_total_trans'])]
#print(clean_data)

In [63]:
from sklearn.preprocessing import LabelEncoder

cat_labels = ['commodity_group_caption_ENG','geoCluster']
data1[cat_labels] = data1[cat_labels].fillna('NaN')

le = LabelEncoder()
label_encoded_df = data1.copy()
for col in cat_labels:
    label_encoded_df[col]=le.fit_transform(label_encoded_df[col])


In [64]:
label_encoded_df

,Unnamed: 0,geoCluster,SKU,date,price,sales,Карантин,USD,Euro,Выборы,productTypeId,brandId,lagerUnitQuantity,lagerUnitTypeId,trademark,commodity_group_caption_ENG
0,1653540,218,507343,2020-01-03,7.19,6.0,0.0,23.6862,26.4220,0,4699.0,8274.0,70.00,1,NaN,0
1,1653541,218,507343,2020-01-08,7.19,4.0,0.0,23.6785,26.3932,0,4699.0,8274.0,70.00,1,NaN,0
2,1045940,218,63037,2020-01-08,14.99,3.0,0.0,23.6785,26.3932,0,3467.0,1241.0,0.50,3,1323.0,2
3,1045989,218,369991,2020-01-08,16.59,2.0,0.0,23.6785,26.3932,0,3465.0,1241.0,0.75,3,1323.0,2
4,1653542,218,507343,2020-01-09,7.19,1.0,0.0,23.8345,26.4909,0,4699.0,8274.0,70.00,1,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1981382,1148294,400,233272,2021-07-19,34.19,2.0,0.0,27.2301,32.1410,0,3454.0,2737.0,1.00,3,1839.0,2
1981383,1974295,481,556482,2021-07-19,11.39,4.0,0.0,27.2301,32.1410,0,4699.0,8274.0,85.00,1,4043.0,0
1981384,1148107,400,67802,2021-07-19,20.79,4.0,0.0,27.2301,32.1410,0,3469.0,1330.0,2.00,3,2781.0,2
1981385,1148508,400,474025,2021-07-19,20.79,7.0,0.0,27.2301,32.1410,0,3470.0,1330.0,2.00,3,2781.0,2


In [65]:
label_encoded_df = label_encoded_df.drop(['Unnamed: 0'], axis=1)
#label_encoded_df = label_encoded_df.drop(['date'], axis=1)

In [66]:
label_encoded_df

,geoCluster,SKU,date,price,sales,Карантин,USD,Euro,Выборы,productTypeId,brandId,lagerUnitQuantity,lagerUnitTypeId,trademark,commodity_group_caption_ENG
0,218,507343,2020-01-03,7.19,6.0,0.0,23.6862,26.4220,0,4699.0,8274.0,70.00,1,NaN,0
1,218,507343,2020-01-08,7.19,4.0,0.0,23.6785,26.3932,0,4699.0,8274.0,70.00,1,NaN,0
2,218,63037,2020-01-08,14.99,3.0,0.0,23.6785,26.3932,0,3467.0,1241.0,0.50,3,1323.0,2
3,218,369991,2020-01-08,16.59,2.0,0.0,23.6785,26.3932,0,3465.0,1241.0,0.75,3,1323.0,2
4,218,507343,2020-01-09,7.19,1.0,0.0,23.8345,26.4909,0,4699.0,8274.0,70.00,1,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1981382,400,233272,2021-07-19,34.19,2.0,0.0,27.2301,32.1410,0,3454.0,2737.0,1.00,3,1839.0,2
1981383,481,556482,2021-07-19,11.39,4.0,0.0,27.2301,32.1410,0,4699.0,8274.0,85.00,1,4043.0,0
1981384,400,67802,2021-07-19,20.79,4.0,0.0,27.2301,32.1410,0,3469.0,1330.0,2.00,3,2781.0,2
1981385,400,474025,2021-07-19,20.79,7.0,0.0,27.2301,32.1410,0,3470.0,1330.0,2.00,3,2781.0,2


In [68]:
import datetime

merged_final = label_encoded_df
new_cols = ['is_monday', 'is_tuesday', 'is_wednesday', 'is_thursday', 'is_friday', 'is_saturday', 'is_sunday']
for index, col_name in enumerate(new_cols):
    merged_final[col_name] = [datetime.datetime.strptime(date_str, '%Y-%m-%d').weekday() == index for date_str in merged_final.date]

In [69]:
merged_final

,geoCluster,SKU,date,price,sales,Карантин,USD,Euro,Выборы,productTypeId,...,lagerUnitTypeId,trademark,commodity_group_caption_ENG,is_monday,is_tuesday,is_wednesday,is_thursday,is_friday,is_saturday,is_sunday
0,218,507343,2020-01-03,7.19,6.0,0.0,23.6862,26.4220,0,4699.0,...,1,NaN,0,False,False,False,False,True,False,False
1,218,507343,2020-01-08,7.19,4.0,0.0,23.6785,26.3932,0,4699.0,...,1,NaN,0,False,False,True,False,False,False,False
2,218,63037,2020-01-08,14.99,3.0,0.0,23.6785,26.3932,0,3467.0,...,3,1323.0,2,False,False,True,False,False,False,False
3,218,369991,2020-01-08,16.59,2.0,0.0,23.6785,26.3932,0,3465.0,...,3,1323.0,2,False,False,True,False,False,False,False
4,218,507343,2020-01-09,7.19,1.0,0.0,23.8345,26.4909,0,4699.0,...,1,NaN,0,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1981382,400,233272,2021-07-19,34.19,2.0,0.0,27.2301,32.1410,0,3454.0,...,3,1839.0,2,True,False,False,False,False,False,False
1981383,481,556482,2021-07-19,11.39,4.0,0.0,27.2301,32.1410,0,4699.0,...,1,4043.0,0,True,False,False,False,False,False,False
1981384,400,67802,2021-07-19,20.79,4.0,0.0,27.2301,32.1410,0,3469.0,...,3,2781.0,2,True,False,False,False,False,False,False
1981385,400,474025,2021-07-19,20.79,7.0,0.0,27.2301,32.1410,0,3470.0,...,3,2781.0,2,True,False,False,False,False,False,False


In [89]:
result = pd.concat([merged_final, pd.get_dummies(merged_final['commodity_group_caption_ENG'])], axis=1)

In [91]:
result = result.drop(['commodity_group_caption_ENG'], axis=1)

In [98]:
result1 = pd.concat([result, pd.get_dummies(merged_final['geoCluster'])], axis=1)
result1 = result1.drop(['geoCluster'], axis=1)

In [80]:
#merged_final = merged_final.drop(['date'], axis=1)
#merged_final = merged_final.fillna(0)
#merged_final = merged_final.drop(['sales'], axis=1)

In [99]:
result1 = result1.fillna(0)

In [165]:
#import statsmodels.api as sm
#est = sm.OLS(y.astype(float), sm.add_constant(result1.astype(float)))
#est2 = est.fit()

#print(est2.summary())
#print('\n\n')
#print(est2.params)

In [ ]:
y_ = est2.predict(sm.add_constant(result1.astype(float)))

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y, y_)

# Cat

In [171]:
X_train

,geoCluster,SKU,price,Карантин,USD,Euro,Выборы,productTypeId,brandId,lagerUnitQuantity,...,is_wednesday,is_thursday,is_friday,is_saturday,is_sunday,0,1,2,3,4
36988,152,32490,21.39,1.0,26.9714,29.2424,0,4754.0,0.0,1.0,...,False,False,True,False,False,0,0,0,1,0
1947807,224,828126,71.09,0.0,27.3047,32.2482,0,383.0,0.0,1.0,...,False,True,False,False,False,1,0,0,0,0
1911446,510,538208,96.59,0.0,27.2862,32.3191,0,4755.0,0.0,1.0,...,False,False,False,True,False,0,0,0,1,0
1972933,436,571805,339.49,0.0,27.2905,32.2369,0,1897.0,7420.0,1.0,...,False,False,False,False,True,0,1,0,0,0
914629,169,775007,17.59,0.0,27.8461,33.6478,0,400.0,7418.0,115.0,...,False,False,False,False,True,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259178,202,810552,241.39,0.0,28.3762,33.4343,0,1894.0,2737.0,1.0,...,False,False,False,False,False,0,1,0,0,0
1414414,400,437495,22.79,1.0,27.7715,33.5577,0,1014.0,8314.0,50.0,...,True,False,False,False,False,1,0,0,0,0
131932,371,596031,36.99,0.0,27.3606,31.2554,0,4698.0,0.0,400.0,...,False,False,False,False,False,1,0,0,0,0
671155,328,612845,49.79,1.0,28.0609,33.9888,0,3282.0,967.0,305.0,...,False,False,False,True,False,0,0,0,0,1


In [122]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(result, y, random_state=42)

from catboost import CatBoostRegressor

model = CatBoostRegressor(iterations=150, depth=15, learning_rate=0.5)
model.fit(X_train, y_train)
y_ = model.predict(X_test)

from sklearn.metrics import mean_absolute_percentage_error
mean_absolute_error(y_test, y_)

0:	learn: 3.2653354	total: 1.5s	remaining: 3m 43s
1:	learn: 3.1478214	total: 2.99s	remaining: 3m 41s
2:	learn: 3.0924155	total: 4.37s	remaining: 3m 34s
3:	learn: 3.0674175	total: 5.81s	remaining: 3m 32s
4:	learn: 3.0450895	total: 7.19s	remaining: 3m 28s
5:	learn: 3.0271329	total: 8.56s	remaining: 3m 25s
6:	learn: 3.0119238	total: 9.97s	remaining: 3m 23s
7:	learn: 2.9837981	total: 11.4s	remaining: 3m 21s
8:	learn: 2.9552405	total: 12.8s	remaining: 3m 19s
9:	learn: 2.9416542	total: 14.2s	remaining: 3m 18s
10:	learn: 2.9343839	total: 15.5s	remaining: 3m 16s
11:	learn: 2.9221164	total: 16.9s	remaining: 3m 14s
12:	learn: 2.8996250	total: 18.2s	remaining: 3m 12s
13:	learn: 2.8944248	total: 19.6s	remaining: 3m 10s
14:	learn: 2.8879510	total: 20.9s	remaining: 3m 8s
15:	learn: 2.8782918	total: 22.3s	remaining: 3m 7s
16:	learn: 2.8741947	total: 23.7s	remaining: 3m 5s
17:	learn: 2.8529538	total: 25.1s	remaining: 3m 4s
18:	learn: 2.8325689	total: 26.5s	remaining: 3m 2s
19:	learn: 2.8278115	total: 

1.4295802185238677

In [136]:
import numpy as np
def mean_absolute_percentage_error_(y_true, y_pred): 
    return np.sum(np.abs(y_true - y_pred) / np.sum(y_true))

In [137]:
mean_absolute_percentage_error_(y_test, y_)

0.4764156681510358

In [226]:
X_train

,geoCluster,SKU,price,Карантин,USD,Euro,Выборы,productTypeId,brandId,lagerUnitQuantity,...,is_wednesday,is_thursday,is_friday,is_saturday,is_sunday,0,1,2,3,4
36988,152,32490,21.39,1.0,26.9714,29.2424,0,4754.0,0.0,1.0,...,False,False,True,False,False,0,0,0,1,0
1947807,224,828126,71.09,0.0,27.3047,32.2482,0,383.0,0.0,1.0,...,False,True,False,False,False,1,0,0,0,0
1911446,510,538208,96.59,0.0,27.2862,32.3191,0,4755.0,0.0,1.0,...,False,False,False,True,False,0,0,0,1,0
1972933,436,571805,339.49,0.0,27.2905,32.2369,0,1897.0,7420.0,1.0,...,False,False,False,False,True,0,1,0,0,0
914629,169,775007,17.59,0.0,27.8461,33.6478,0,400.0,7418.0,115.0,...,False,False,False,False,True,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259178,202,810552,241.39,0.0,28.3762,33.4343,0,1894.0,2737.0,1.0,...,False,False,False,False,False,0,1,0,0,0
1414414,400,437495,22.79,1.0,27.7715,33.5577,0,1014.0,8314.0,50.0,...,True,False,False,False,False,1,0,0,0,0
131932,371,596031,36.99,0.0,27.3606,31.2554,0,4698.0,0.0,400.0,...,False,False,False,False,False,1,0,0,0,0
671155,328,612845,49.79,1.0,28.0609,33.9888,0,3282.0,967.0,305.0,...,False,False,False,True,False,0,0,0,0,1


# Predict

In [234]:
import pandas as pd
test_data = pd.read_csv('test_data.csv')

In [235]:
idd = test_data['ID']
test_data = test_data.drop(['ID'], axis=1)
#test_data = test_data.drop(['geoCluster'], axis=1)

In [236]:
import pandas as pd
data_dollar = pd.read_excel('dollar.xlsx')[1:]
data_dollar.dropna(subset = ["Euro"], inplace=True)
data_dollar=data_dollar.fillna(0)
data_dollar=data_dollar.rename(columns={'Data': 'date'})
data_dollar['date'] = data_dollar['date'].astype(str)
outer_merged = pd.merge(test_data, data_dollar, how="left", on=["date","date"])

In [237]:
import pandas as pd
sku_final = pd.read_csv('sku_final.csv')
merged_final = pd.merge(outer_merged, sku_final, how="left", on=["SKU","SKU"])

In [238]:
merged_final = merged_final.drop(['commodity_group_caption_RU','commodity_group_caption_UKR','productType_caption_RU','productType_caption_UKR','productCategory_caption_UKR','productCategory_caption_RU'], axis=1)
merged_final = merged_final.drop(['countryOfOrigin','productType_caption_ENG','Year','lagerUnitType_caption'], axis=1)
merged_final = merged_final.drop(['commodity_group','productCategoryId'], axis=1)
merged_final = merged_final.drop(['productCategory_caption_ENG'], axis=1)

In [239]:
merged_final = merged_final.drop(['countryOfOrigin_caption'], axis=1)

In [240]:
merged_final = merged_final.sort_values(by=['date'])

In [241]:
import datetime

new_cols = ['is_monday', 'is_tuesday', 'is_wednesday', 'is_thursday', 'is_friday', 'is_saturday', 'is_sunday']
for index, col_name in enumerate(new_cols):
    merged_final[col_name] = [datetime.datetime.strptime(date_str, '%Y-%m-%d').weekday() == index for date_str in merged_final.date]

In [242]:
result_ = pd.concat([merged_final, pd.get_dummies(merged_final['commodity_group_caption_ENG'])], axis=1)
result_ = result_.drop(['commodity_group_caption_ENG','date'], axis=1)

In [243]:
result_

,geoCluster,SKU,price,Карантин,USD,Euro,Выборы,productTypeId,brandId,lagerUnitQuantity,...,is_wednesday,is_thursday,is_friday,is_saturday,is_sunday,Bakery,Cheese,Mineral water,Tropical fruits,Yogurts
0,21,32485,66.69,0.0,27.2132,32.0313,0,4752.0,NaN,1.0,...,False,False,False,False,False,0,0,0,1,0
1651034,3168,775006,36.69,0.0,27.2132,32.0313,0,400.0,7418.0,115.0,...,False,False,False,False,False,0,0,0,0,1
209888,2019,719934,268.49,0.0,27.2132,32.0313,0,1895.0,2089.0,1.0,...,False,False,False,False,False,0,1,0,0,0
1026746,2246,49215,283.99,0.0,27.2132,32.0313,0,4758.0,NaN,1.0,...,False,False,False,False,False,0,0,0,1,0
1026732,2246,49207,283.99,0.0,27.2132,32.0313,0,4768.0,NaN,1.0,...,False,False,False,False,False,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1036713,2248,566113,9.99,0.0,26.8168,31.8946,0,3104.0,1594.0,36.0,...,False,False,False,False,False,0,0,0,0,1
1036727,2248,566929,9.99,0.0,26.8168,31.8946,0,1897.0,6666.0,1.0,...,False,False,False,False,False,0,1,0,0,0
1036741,2248,569082,9.99,0.0,26.8168,31.8946,0,3283.0,967.0,300.0,...,False,False,False,False,False,0,0,0,0,1
1036587,2248,555470,18.49,0.0,26.8168,31.8946,0,3104.0,1594.0,36.0,...,False,False,False,False,False,0,0,0,0,1


In [203]:
y_ans = model.predict(result_)

df_answer = pd.DataFrame(columns=['ID','sales'])
df_answer['sales'] = y_ans
df_answer['ID'] = idd

df_answer

df_answer.to_csv('out.csv',index=False)

In [183]:
df_answer.to_csv('out.csv',index=False)